In [1]:
import sys
sys.path.insert(0, '/das/work/p17/p17983/')

In [2]:
%matplotlib nbagg
import jungfrau_utils as ju
import numpy as np
import json
import os
import time
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

import h5py

from alvra_tools.load_data import *
from alvra_tools.channels import *
from built_functions import *

[INFO][corrections] Numba not available, reverting to Numpy


In [3]:
def get_xes_pumped(filename, DIR, DIRBS, roi, channel, ynstamp, N_las, N_x):
    numstds = 2.5
    minIzero = 0.01
    lin_filter = 0.2
    image_threshold = 1.5
    hot_pixel = 10

    print(DIR + filename + ".JF02T09V02crop.h5")
    print()
    print(DIRBS + filename + ".BSREAD.h5")
    
    images_off, images_on, pulse_ids_off, pulse_ids_on = \
        load_JF_cropped_data_pump(DIR + filename + ".JF02T09V02crop.h5", roi, 50, 50/(N_las + N_x), nshots=None)
    (DataFluo_pump, DataFluo_unpump, IzeroFEL_pump, IzeroFEL_unpump, JFIDs_pump, JFIDs_unpump, Energy_pump, Energy_unpump, _, _) = \
        load_PumpProbe_events_withTwoVariables(DIRBS + filename + ".BSREAD.h5", channel_BS_pulse_ids, channel)
    (_, _, _, _, _, _, Laser_pump, Laser_unpump, _, _) = \
        load_PumpProbe_events_withTwoVariables(DIRBS + filename + ".BSREAD.h5", channel_PIPS_fluo, channel)
    
    Energy_pump = [x for x in Energy_pump if (np.abs(x) > 0)]
    Energy_eV_pump = np.mean(Energy_pump)
    
    Energy_unpump = [x for x in Energy_unpump if (np.abs(x) > 0)]
    Energy_eV_unpump = np.mean(Energy_unpump)
    
    idcheck = [a==b and c==d for a,b,c,d in zip(pulse_ids_on, JFIDs_pump, pulse_ids_off, JFIDs_unpump)]
    IDCheck = sum([int(x) for x in idcheck])
    
    if IDCheck:
        print("passed ID check")
    else:
        print("failed ID check")   
    
    IzeroMedian = np.median(np.concatenate([IzeroFEL_pump, IzeroFEL_unpump]))
    IzeroSTD = np.std(np.concatenate([IzeroFEL_pump, IzeroFEL_unpump]))
    
    print('numon')
    print(IzeroFEL_pump.shape[0])
    
    print('numoff')
    print(IzeroFEL_unpump.shape[0])
    
    print('number of frames')
    print(images_on.shape[0])
    print(images_off.shape[0])

    images_good_on = images_on
    images_good_off = images_off

    images_thr_on = images_good_on.copy()
    images_thr_on[images_good_on < image_threshold] = 0
    images_thr_on[images_good_on > hot_pixel] = 0
    images_thr_on[np.isnan(images_thr_on)] = 0
    
    images_thr_off = images_good_off.copy()
    images_thr_off[images_good_off < image_threshold] = 0
    images_thr_off[images_good_off > hot_pixel] = 0
    images_thr_off[np.isnan(images_thr_off)] = 0
    
    print('num nan')
    print(sum(sum(sum(np.isnan(images_thr_on)))))
    
    print('Izero')
    print(np.mean(IzeroFEL_unpump))
    print(np.mean(IzeroFEL_pump))
    print(np.max(IzeroFEL_unpump))
    print(np.min(IzeroFEL_unpump))
    
    IzeroFEL_pump, IzeroFEL_unpump, DataFluo_pump, DataFluo_unpump, condFinalPump, condFinalUnPump = \
        filterData(IzeroFEL_pump, IzeroFEL_unpump, DataFluo_pump, DataFluo_unpump, numstds, minIzero, lin_filter)
    

    
    print(images_thr_on.shape)
    print(images_thr_on.shape)
    
    images_thr_on = images_thr_on[condFinalPump]
    images_thr_off = images_thr_off[condFinalUnPump]

    print('number of surviving frames')
    print(images_thr_on.shape[0])
    print(images_thr_off.shape[0])
    
    XES_on = images_thr_on.sum(axis=0)/images_thr_on.shape[0]
    XES_off = images_thr_off.sum(axis=0)/images_thr_off.shape[0]
    
    XES_on[np.isnan(XES_on)] = 0
    XES_off[np.isnan(XES_off)] = 0
    
    print('num nan')
    print(sum(sum(np.isnan(XES_on))))
    
    stamp = make_bar_stamp(XES_on.shape[1],XES_on.shape[0])
    
    print(XES_on.shape)
    print(stamp.shape)
    if ynstamp:

        XES_on = XES_on * stamp
        XES_off = XES_off * stamp
        
    FloFEL_pump_total = np.mean(DataFluo_pump)/np.mean(IzeroFEL_pump)
    FloFEL_unpump_total = np.mean(DataFluo_unpump)/np.mean(IzeroFEL_unpump)
    
    return XES_on, XES_off, Energy_eV_pump, Energy_eV_unpump, FloFEL_pump_total, FloFEL_unpump_total


In [4]:
def make_bar_stamp(dimension_x, dimension_y):

    X,Y = np.meshgrid(np.linspace(0,dimension_x-1,dimension_x),np.linspace(0,dimension_y-1,dimension_y))

    Z = np.zeros([dimension_y,dimension_x])+1

    condition = np.logical_and(Y<260, Y>253)
    Z[condition] = 0
    
    Z[Y>320] = 0
    Z[Y < 200] = 0

    condition = np.logical_not(np.logical_and(X<200, X>130))
    Z[condition] = 0

    #plt.figure()
    #plt.imshow(Z)
    
    #plt.colorbar()
    
    return Z

Run for RIXS

In [5]:
def make_rixs_planes(scannum, base, numfiles, roi, N_las, N_x):
    
    for jj in range(len(scannum)):

        
        filename_base = base + '%02d' % scannum[jj] + "_step00"
        scan_name = base + '%02d' % scannum[jj]
        DIR = "/das/work/p17/p17983/cropped_data/scan_data/" + scan_name + "/"
        DIRBS = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name + "/"

#         filename_base = base + '%02d' % scannum[jj] + "_step00"
#         scan_name = base + '%02d' % scannum[jj]
#         DIR = "/das/work/p17/p17983/cropped_data/scan_data/" + scan_name + "/"
#         DIRBS = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name + "/"


    #     filename_base = "RuBpy3_monoscan_0" + '%02d' % scannum[jj] + "_step00"
    #     scan_name = "RuBpy3_monoscan_0" + '%02d' % scannum[jj]
    #     DIR = "/das/work/p17/p17983/cropped_data/scan_data/" + scan_name + "/"
    #     DIRBS = "/sf/alvra/data/p17983/raw/scan_data/" + scan_name + "/"
    #     numfiles = 35+1

        eV_on = np.empty(numfiles)
        eV_off = np.empty(numfiles)

        fluo_Pump = np.empty(numfiles)
        fluo_Unpump = np.empty(numfiles)


        for ii in range(numfiles):

            XES_on, XES_off, Energy_eV_pump, Energy_eV_unpump, fluo_pump, fluo_unpump = \
                get_xes_pumped(filename_base + '%02d' % (ii+0), DIR, DIRBS, roi, channel_energy, True, N_las, N_x)

            if False:

                plt.figure()
                plt.subplot(1,2,1)
                plt.imshow(XES_on, vmax = 0.001)
                plt.colorbar()
                plt.subplot(1,2,2)
                plt.imshow(XES_off, vmax = 0.001)
                plt.colorbar()
                plt.tight_layout()


            if ii == 0:
                rixs_on = XES_on.sum(axis=0)
                rixs_off = XES_off.sum(axis=0)
            else:
                rixs_on = np.vstack((rixs_on,XES_on.sum(axis=0)))
                rixs_off = np.vstack((rixs_off,XES_off.sum(axis=0)))

            eV_on[ii] = Energy_eV_pump
            eV_off[ii] = Energy_eV_unpump
            fluo_Pump[ii] = fluo_pump
            fluo_Unpump[ii] = fluo_unpump

        if jj == 0:

            RIXS_on = rixs_on
            RIXS_off = rixs_off
            Fluo_pump = fluo_Pump
            Fluo_unpump = fluo_Unpump
            NumJ = 0

        else:

            RIXS_on = RIXS_on + rixs_on
            RIXS_off = RIXS_off + rixs_off
            Fluo_pump = Fluo_pump + fluo_Pump
            Fluo_unpump = Fluo_unpump + fluo_Unpump
            NumJ = NumJ+jj
            
    print(NumJ)

    return eV_on, eV_off, RIXS_on, RIXS_off, Fluo_pump, Fluo_unpump

In [6]:
def save_data(save_name_base, scannum):
    save_name = save_name_base
    for jj in range(0, len(scannum)):
        save_name = save_name + '%02d' % scannum[jj] + '_'
    
    np.save(save_name + 'eV_on.npy', eV_on)
    print(save_name + 'eV_on.npy' + ' saved!')
    np.save(save_name + 'eV_off.npy', eV_off)
    print(save_name + 'eV_off.npy' + ' saved!')
    np.save(save_name + 'RIXS_on.npy', RIXS_on)
    print(save_name + 'RIXS_on.npy' + ' saved!')
    np.save(save_name + 'RIXS_off.npy', RIXS_off)
    print(save_name + 'RIXS_off.npy' + ' saved!')
    np.save(save_name + 'Fluo_pump.npy', Fluo_pump)
    print(save_name + 'Fluo_pump.npy' + ' saved!')
    np.save(save_name + 'Fluo_unpump.npy', Fluo_unpump)
    print(save_name + 'Fluo_unpump.npy' + ' saved!')    
   

In [7]:
def load_data(load_name_base, scannum):
    load_name = load_name_base
    
    for jj in range(0, len(scannum)):
        load_name = load_name + '%02d' % scannum[jj] + '_'
    
    eV_on = np.load(load_name + 'eV_on.npy')
    eV_off = np.load(load_name + 'eV_off.npy')
    RIXS_on = np.load(load_name + 'RIXS_on.npy')
    RIXS_off = np.load(load_name + 'RIXS_off.npy')
    Fluo_pump = np.load(load_name + 'Fluo_pump.npy')
    Fluo_unpump = np.load(load_name + 'Fluo_unpump.npy')
    
    return eV_on, eV_off, RIXS_on, RIXS_off, Fluo_pump, Fluo_unpump

In [8]:
saveON = 1

# #---------------  RuDimerACN, 0.6 ps, horz pol
# scannum = [6, 7, 9]
# base = "RuDimerACN_monoscan_0p6ps_0"
# numfiles = 54+1
# SBASE = '/das/work/p17/p17983/Reduced_data/RuDimerACN_0p6ps_scans_'

# #---------------  RuDimerACN, 10 ps, horz pol
# scannum = [5, 8, 10, 13]
# base = "RuDimerACN_monoscan_10ps_0"
# numfiles = 54+1
# SBASE = '/das/work/p17/p17983/Reduced_data/RuDimerACN_10ps_scans_'
# roi = 2
# N_las = 2
# N_x = 1

# #--------------- RuDimerACN, 0.6 ps, vert pol
# scannum = [10, 11, 12]
# base = "RuDimerACN_monoscan_0p6ps_0"
# numfiles = 54+1
# SBASE = '/das/work/p17/p17983/Reduced_data/RuDimerACN_0p6ps_scans_'

# #--------------- RuDimerACN, 10 ps, vert pol
# scannum = [15, 16, 17, 18]
# base = "RuDimerACN_monoscan_10ps_0"
# numfiles = 54+1
# SBASE = '/das/work/p17/p17983/Reduced_data/RuDimerACN_10ps_scans_'

# #--------------- RuDimerCl, 0.6 ps, horz pol, 2 uJ
# scannum = [1, 4, 5]
# base = "RuDimerCl_monoscan_0p6ps_0"
# numfiles = 54+1
# SBASE = '/das/work/p17/p17983/Reduced_data/RuDimerCl_0p6ps_scans_'
# roi = 2
# N_las = 1
# N_x = 1

# #--------------- RuDimerCl, 0.6 ps, horz pol, 0.5 uJ, not centered on jet, possibly sample degrad
# scannum = [6]
# base = "RuDimerCl_monoscan_0p6ps_0"
# numfiles = 54+1
# SBASE = '/das/work/p17/p17983/Reduced_data/RuDimerCl_0p6ps_scans_'
# roi = 2
# N_las = 1
# N_x = 1

#--------------- RuDimerCl, 0.6 ps, horz pol, 0.5 uJ, not centered on jet, possibly sample degrad, shorter energy range
scannum = [10, 11, 12]
base = "RuDimerCl_monoscan_0p6ps_0"
numfiles = 28+1
SBASE = '/das/work/p17/p17983/Reduced_data/RuDimerCl_0p6ps_scans_'
roi = 2
N_las = 1
N_x = 1










eV_on, eV_off, RIXS_on, RIXS_off, Fluo_pump, Fluo_unpump = make_rixs_planes(scannum, base, numfiles, roi, N_las, N_x)

if saveON:
    save_data(SBASE, scannum)


eV_on, eV_off, RIXS_on, RIXS_off, Fluo_pump, Fluo_unpump = make_rixs_planes(scannum, base, numfiles, roi, N_las, N_x)

if saveON:
    save_data(SBASE, scannum)


/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_010/RuDimerCl_monoscan_0p6ps_010_step0000.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_010/RuDimerCl_monoscan_0p6ps_010_step0000.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06681280837766826
0.06586045882850886
0.15395250916481018
0.015147640369832516
(500, 512, 300)
(500, 512, 300)
number of surviving frames
490
489
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_010/RuDimerCl_monoscan_0p6ps_010_step0001.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_010/RuDimerCl_monoscan_0p6ps_010_step0001.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06752253

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06713676133006811
0.06457937400788069
0.20243754982948303
0.020666178315877914
(500, 512, 300)
(500, 512, 300)
number of surviving frames
486
486
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_010/RuDimerCl_monoscan_0p6ps_010_step0015.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_010/RuDimerCl_monoscan_0p6ps_010_step0015.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06873892006650567
0.06839585408195853
0.1869572401046753
0.015646476298570633
(500, 512, 300)
(500, 512, 300)
number of surviving frames
487
486
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_010/RuDimerCl_mon

num nan
0
Izero
0.06473420021310448
0.06564025888592005
0.1762094795703888
0.01775662787258625
(500, 512, 300)
(500, 512, 300)
number of surviving frames
488
487
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0000.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0000.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06838165653118085
0.06776834416948259
0.1701851338148117
1.3956768270872999e-05
(500, 512, 300)
(500, 512, 300)
number of surviving frames
487
485
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0001.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_

number of surviving frames
485
490
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0014.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0014.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06474169298633933
0.06572529535740614
0.16207288205623627
0.018176747485995293
(500, 512, 300)
(500, 512, 300)
number of surviving frames
478
488
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0015.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0015.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06571986246854067
0.06679659466259182
0.1586574912071228
-0.000916600227355957
(500, 512, 300)
(500, 512, 300)
number of surviving frames
488
488
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_012/RuDimerCl_monoscan_0p6ps_012_step0000.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_012/RuDimerCl_monoscan_0p6ps_012_step0000.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06642897582054139
0.06757224265113473
0.16781172156333923
0.019799083471298218
(500, 512, 300)
(500, 512, 300)
number of surviving frames
480
486
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_012/RuDimerCl_mo

...

num nan
0
Izero
0.06520557551458478
0.06646230655536056
0.1441173404455185
0.023801574483513832
(500, 512, 300)
(500, 512, 300)
number of surviving frames
484
485
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_010/RuDimerCl_monoscan_0p6ps_010_step0027.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_010/RuDimerCl_monoscan_0p6ps_010_step0027.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06450831020623446
0.06697460130229592
0.1720176339149475
0.020510233938694
(500, 512, 300)
(500, 512, 300)
number of surviving frames
487
489
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_010/RuDimerCl_monoscan_0p6ps_010_step0028.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_010/RuDimerCl_monoscan_0p6ps_010_

number of surviving frames
483
486
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0012.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0012.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06768073983117938
0.06555599415302277
0.15787732601165771
0.023709379136562347
(500, 512, 300)
(500, 512, 300)
number of surviving frames
489
482
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0013.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0013.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon

SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06564925339072943
0.06548253006115556
0.153779998421669
0.01973293349146843
(500, 512, 300)
(500, 512, 300)
number of surviving frames
488
482
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0027.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monoscan_0p6ps_011_step0027.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06591233760304749
0.06582369839772582
0.15185604989528656
0.014211938716471195
(500, 512, 300)
(500, 512, 300)
number of surviving frames
485
483
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_011/RuDimerCl_monos

num nan
0
Izero
0.06742451516538858
0.06748906700313091
0.16160166263580322
0.014115963131189346
(500, 512, 300)
(500, 512, 300)
number of surviving frames
486
485
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_012/RuDimerCl_monoscan_0p6ps_012_step0012.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_012/RuDimerCl_monoscan_0p6ps_012_step0012.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06515326527133584
0.0662685084361583
0.20260214805603027
0.01567794755101204
(500, 512, 300)
(500, 512, 300)
number of surviving frames
491
488
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_012/RuDimerCl_monoscan_0p6ps_012_step0013.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_012/RuDimerCl_monoscan_0p6ps_0

number of surviving frames
484
484
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_012/RuDimerCl_monoscan_0p6ps_012_step0026.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_012/RuDimerCl_monoscan_0p6ps_012_step0026.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
500
numoff
500
number of frames
500
500
num nan
0
Izero
0.06735719047486782
0.0674395534209907
0.18936003744602203
0.019619371742010117
(500, 512, 300)
(500, 512, 300)
number of surviving frames
490
481
num nan
0
(512, 300)
(512, 300)
/das/work/p17/p17983/cropped_data/scan_data/RuDimerCl_monoscan_0p6ps_012/RuDimerCl_monoscan_0p6ps_012_step0027.JF02T09V02crop.h5

/sf/alvra/data/p17983/raw/scan_data/RuDimerCl_monoscan_0p6ps_012/RuDimerCl_monoscan_0p6ps_012_step0027.BSREAD.h5
images_roi2
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
SLAAR11-LMOT-M451:ENC_1_BS/pulse_id
passed ID check
numon
